In [4]:
import os
import csv

# Define the directory containing the data folders
data_directory = "/Users/franceskoback/Downloads/RK.E0CC9B64.Wearable Wellness_20230515-20230516"
data_directory = "/Users/franceskoback/Downloads/RK.E0CC9B64.Wearable Wellness_20230522-20230523"
data_directory = "/Users/franceskoback/Downloads/RK.E0CC9B64.Wearable Wellness_20230527-20230528"
data_directory = "/Users/franceskoback/Downloads/RK.E0CC9B64.Wearable Wellness_20230526-20230527"

# Define the output file path for the aggregate CSV
output_file = "aggregate.csv"

# Define the columns to extract from each device's CSV
columns = ["ParticipantIdentifier", "Date", "RestingHeartRate", "Steps", "Sleep"]

# Initialize the aggregate data dictionary
aggregate_data = []

# Iterate over the data folders
csv_file = [file for file in os.listdir(data_directory) if file.startswith("AllParticipants") and file.endswith(".csv")]
csv_file = os.path.join(data_directory, csv_file[0])

with open(csv_file, "r", encoding="utf-8-sig") as f:
    reader = csv.reader(f)
    header_row = next(reader)
    header_row = [column.strip('\ufeff') for column in header_row]
    participant_identifier_index = header_row.index("ParticipantIdentifier")
    secondary_identifier_index = header_row.index("SecondaryIdentifier")

    for row in reader:
        participant_identifier = row[participant_identifier_index]
        device_type = row[secondary_identifier_index]

        if device_type:
            participant_date = row[1].split("T")[0]  # Extract the date portion

        if device_type and device_type.startswith("Apple"):
            # Search for resting heart rate CSV
            resting_heart_rate_file = [file for file in os.listdir(data_directory) if
                                       file.startswith("HealthKitV2Samples_RestingHeartRate_") and file.endswith(".csv")]
            if resting_heart_rate_file:
                resting_heart_rate_file = os.path.join(data_directory, resting_heart_rate_file[0])
                with open(resting_heart_rate_file, "r") as resting_hr_csv:
                    resting_hr_reader = csv.reader(resting_hr_csv)
                    next(resting_hr_reader)  # Skip the header row
                    resting_heart_rate_date = None  # Initialize the resting heart rate date
                    for hr_row in resting_hr_reader:
                        if hr_row[1] == participant_identifier:
                            resting_heart_rate = hr_row[4]
                            resting_heart_rate_date = hr_row[3].split("T")[0]  # Extract the date portion
                            break

                # Check if resting heart rate data is found
                if resting_heart_rate_date:
                    # Create a new row for the participant and date
                    new_row = [participant_identifier, resting_heart_rate_date, resting_heart_rate, None, None]

                    # Add the new row to the aggregate_data list
                    aggregate_data.append(new_row)

            # Search for daily steps CSV
            daily_steps_file = [file for file in os.listdir(data_directory) if
                                file.startswith("HealthKitV2Statistics_DailySteps_") and file.endswith(".csv")]
            if daily_steps_file:
                daily_steps_file = os.path.join(data_directory, daily_steps_file[0])
                with open(daily_steps_file, "r") as daily_steps_csv:
                    daily_steps_reader = csv.reader(daily_steps_csv)
                    next(daily_steps_reader)  # Skip the header row

                    # Continue with the remaining steps of the code

                    daily_steps_date= None
                    daily_steps= None
                    for steps_row in daily_steps_reader:
                        if steps_row[1] == participant_identifier:
                            daily_steps = steps_row[4]
                            daily_steps_date = steps_row[3].split("T")[0]  # Extract the date portion
                            break

                
                # Check if daily steps data is found
                if daily_steps_date:
                    # Create a new row for the participant and date
                    new_row = [participant_identifier, daily_steps_date, None, daily_steps, None]
                    # Add the new row to the aggregate_data list
                    aggregate_data.append(new_row)

                

       
        elif device_type.startswith("Fitbit"):
            # Search for Fitbit daily data CSV
            fitbit_data_file = [file for file in os.listdir(data_directory) if
                                file.startswith("FitbitDailyData") and file.endswith(".csv")]
            if fitbit_data_file:
                fitbit_data_file = os.path.join(data_directory, fitbit_data_file[0])
                with open(fitbit_data_file, "r") as fitbit_data_csv:
                    fitbit_data_reader = csv.reader(fitbit_data_csv)
                    next(fitbit_data_reader)  # Skip the header row
                    for fitbit_row in fitbit_data_reader:
                        if fitbit_row[0] == participant_identifier:
                            fitbit_date = fitbit_row[1].split("T")[0]  # Extract the date portion
                            steps = fitbit_row[40]
                            resting_heart_rate = fitbit_row[39]

                            # Update the corresponding date's row in the aggregate_data list
                            for data_row in aggregate_data:
                                if data_row[0] == participant_identifier and data_row[1] == fitbit_date:
                                    data_row[2] = str(int(data_row[2]) + int(resting_heart_rate))
                                    data_row[3] = str(int(data_row[3]) + int(steps))
                                    break
                            else:
                                # Create a new row for the participant and date
                                new_row = [participant_identifier, fitbit_date, resting_heart_rate, steps, None]
                                aggregate_data.append(new_row)

            # Search for Fitbit sleep logs CSV
            fitbit_sleep_file = [file for file in os.listdir(data_directory) if
                                  file.startswith("FitbitSleepLogs") and file.endswith(".csv")]
            if fitbit_sleep_file:
                fitbit_sleep_file = os.path.join(data_directory, fitbit_sleep_file[0])
                with open(fitbit_sleep_file, "r") as fitbit_sleep_csv:
                    fitbit_sleep_reader = csv.reader(fitbit_sleep_csv)
                    next(fitbit_sleep_reader)  # Skip the header row
                    for sleep_row in fitbit_sleep_reader:
                        if sleep_row[0] == participant_identifier:
                            sleep_date = sleep_row[2].split("T")[0]  # Extract the date portion
                            sleep_duration = int(sleep_row[7]) / 60  # Convert minutes to hours

                            # Update the corresponding date's row in the aggregate_data list
                            for data_row in aggregate_data:
                                if data_row[0] == participant_identifier and data_row[1] == sleep_date:
                                    data_row[4] = str(sleep_duration)
                                    break
                            else:
                                # Create a new row for the participant and date
                                new_row = [participant_identifier, sleep_date, None, None, str(sleep_duration)]
                                aggregate_data.append(new_row)
                                break

        elif device_type.startswith("Garmin"):
            # Search for Garmin daily summary CSV files
            garmin_summary_files = [file for file in os.listdir(data_directory) if
                                    file.startswith("GarminDailySummary_") and "samples" not in file]
            for garmin_summary_file in garmin_summary_files:
                garmin_summary_file = os.path.join(data_directory, garmin_summary_file)
                with open(garmin_summary_file, "r") as garmin_summary_csv:
                    garmin_summary_reader = csv.reader(garmin_summary_csv)
                    next(garmin_summary_reader)  # Skip the header row
                    for garmin_row in garmin_summary_reader:
                        if garmin_row[1] == participant_identifier:
                            garmin_date = garmin_row[6].split("T")[0]  # Extract the date portion
                            resting_heart_rate = garmin_row[18]
                            steps = garmin_row[9]

                            # Update the corresponding date's row in the aggregate_data list
                            for data_row in aggregate_data:
                                if data_row[0] == participant_identifier and data_row[1] == garmin_date:
                                    data_row[2] = str(int(data_row[2] or 0) + int(resting_heart_rate or 0))
                                    data_row[3] = str(int(data_row[3] or 0) + int(steps or 0))
                                    #break
                            else:
                                # Create a new row for the participant and date
                                new_row = [participant_identifier, garmin_date, resting_heart_rate, steps, None]
                                aggregate_data.append(new_row)

                            # Search for Garmin sleep summary CSV
                            garmin_sleep_files = [file for file in os.listdir(data_directory) if
                                                  file.startswith("GarminSleepSummary_") and file.endswith(".csv")]
                            for garmin_sleep_file in garmin_sleep_files:
                                garmin_sleep_file = os.path.join(data_directory, garmin_sleep_file)
                                with open(garmin_sleep_file, "r") as garmin_sleep_csv:
                                    garmin_sleep_reader = csv.reader(garmin_sleep_csv)
                                    next(garmin_sleep_reader)  # Skip the header row
                                    for sleep_row in garmin_sleep_reader:
                                        if sleep_row[1] == participant_identifier:
                                            sleep_date = sleep_row[6].split("T")[0]  # Extract the date portion
                                            sleep_duration = int(sleep_row[5]) / 60 / 60  # Convert seconds to hours

                                            # Update the corresponding date's row in the aggregate_data list
                                            for data_row in aggregate_data:
                                                if data_row[0] == participant_identifier and data_row[1] == sleep_date:
                                                    data_row[4] = str(sleep_duration)
                                                    break
                                            else:
                                                # Create a new row for the participant and date
                                                new_row = [participant_identifier, sleep_date, None, None, str(sleep_duration)]
                                                aggregate_data.append(new_row)
                                                break

# Sort the aggregate_data list by date
aggregate_data.sort(key=lambda x: x[1])

# Print the extracted data
for data_row in aggregate_data:
    print(data_row)

# Write the aggregate data to the output CSV file
with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(columns)
    writer.writerows(aggregate_data)

['MDH-0806-2798', '2023-05-18', '73', '10206', '6.95']
['MDH-0806-2798', '2023-05-19', '73', '13133', '6.333333333333333']
['MDH-0806-2798', '2023-05-20', '73', '22670', '7.283333333333333']
['MDH-0806-2798', '2023-05-21', '73', '28489', '6.533333333333333']
['MDH-0806-2798', '2023-05-22', '73', '8503', '7.416666666666667']
['MDH-0806-2798', '2023-05-23', '75', '12609', '7.233333333333333']
['MDH-0806-2798', '2023-05-24', '77', '10462', '5.783333333333333']
['MDH-0806-2798', '2023-05-25', '78', '14245', '6.383333333333334']
['MDH-9789-3303', '2023-05-25', '42', '4500', '6.1']
['MDH-9789-3303', '2023-05-25', '42', '4500', None]
['MDH-0806-2798', '2023-05-26', '79', '133', '6.7']
['MDH-9789-3303', '2023-05-26', '53', '3830', None]
